In [1]:
from dotenv import load_dotenv
import json
import os

import numpy as np
import pandas as pd
from pyairtable import Api
import requests


from calitp_data_analysis.sql import to_snakecase
import script

In [2]:
load_dotenv()
MTC_API_KEY = os.getenv('MTC_API_KEY')
api = Api(os.getenv('AIRTABLE_TOKEN'))
mtc_operator_url = f"https://api.511.org/transit/operators?api_key={MTC_API_KEY}"

In [15]:
pd.options.display.max_columns = 100

In [3]:

# pd.options.display.float_format = "{:.2f}".format
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_colwidth", None)

In [4]:
# Trying to stay consistent with 
# https://github.com/cal-itp/data-infra/blob/main/airflow/plugins/operators/airtable_to_gcs.py
def all_rows_as_df(base_id, table_name):
    all_rows = api.table(base_id=base_id, table_name=table_name).all()

    df = pd.DataFrame(
        [
            {"id":row["id"], **row["fields"]}
            for row in all_rows
        ]
    )
    return df

def takeout_list(x):
    if x is not np.nan:
        return x[0]


In [5]:
TABLE_TRANSIT_DATA_QUALITY_ISSUES_ID = 'appmBGOFTvsDv4jdJ'

In [6]:
issues_df = all_rows_as_df(TABLE_TRANSIT_DATA_QUALITY_ISSUES_ID, 'tblEv7QTfEmypU6gg')
issue_types = all_rows_as_df(TABLE_TRANSIT_DATA_QUALITY_ISSUES_ID, 'tblupkIe04LxEPWSR')
services_df = all_rows_as_df(TABLE_TRANSIT_DATA_QUALITY_ISSUES_ID, 'tblBZtbuntv4D0i1u')

In [7]:
#Clean up columns where data is a bunch of single item lists
issues_df['Issue Type'] = issues_df['Issue Type'].apply(takeout_list)
issues_df['Services'] = issues_df['Services'].apply(takeout_list) 

In [8]:
partial = pd.merge(issues_df,issue_types[['id','Name','Notes']],how='left',left_on='Issue Type', right_on='id').drop(columns=['id_x','Issue Type','Created By', 'Last Modified', 'Last Modified By']).rename(columns={'Name':'Issue Type'})

In [9]:
df = pd.merge(partial, services_df[['id','Name']], how='left', left_on='Services', right_on='id').drop(columns=['Services','id_y']).rename(columns={'Name':'Service Name'})
# df.head()

In [17]:
df.head()

,Description,GTFS Datasets,Status,Issue #,Resolution Date,Assignee,Issue Creation Time,Waiting over a week?,QC: Num services,QC: Num Issue Types,QC Checks,Waiting on someone other than transit data quality?,Caltrans District (from Operating County Geographies) (from Services),Is Open,Last Update Month,Last Update Year,Status Notes,Waiting Since,Outreach Status,Should wait until,Issue Type,Notes,id,Service Name
0,Palo Verde Valley Transit Agency: GTFS Schedul...,[recnCtfmZcTSsrJh7],Fixed - with Cal-ITP help,21,2023-05-19,"{'id': 'usr2fo9oGEjsugXf0', 'email': 'evan@cal...",2023-05-25T00:16:26.000Z,N/A,1,1,OK,No,[8],No,5,2023,NaN,NaN,NaN,NaN,Expired Schedule Feed,NaN,recVmX1bV8LrzyGl7,Desert Roadrunner
1,City of Auburn: GTFS Schedule Feed has Expired,[recqOFeSKMAP6tcGp],Fixed - with Cal-ITP help,11,2023-04-13,"{'id': 'usr2fo9oGEjsugXf0', 'email': 'evan@cal...",2023-05-25T00:16:26.000Z,N/A,1,1,OK,No,[3],No,5,2023,NaN,NaN,NaN,NaN,Expired Schedule Feed,NaN,recAk8l02ipaRB34a,Auburn Transit
2,The GTFS schedule feed is about to expire on 1...,[recoOHimkMbZlH3vV],Fixed - on its own,143,2024-01-03,"{'id': 'usrnOT3BlbNIbiG9v', 'email': 'md.islam...",2023-12-29T00:28:14.000Z,N/A,1,1,OK,No,[2],No,1,2024,As of 12/28: The GTFS schedule feed is about t...,NaN,NaN,NaN,About to Expire Schedule Feed,NaN,reczR85P0abN3zb3n,Trinity Transit
3,Los Angeles County Metropolitan Transportation...,[recVvXBJDgsRqVLy6],Fixed - with Cal-ITP help,47,2023-06-29,"{'id': 'usr2fo9oGEjsugXf0', 'email': 'evan@cal...",2023-05-25T00:16:26.000Z,N/A,1,1,OK,No,[7],No,9,2023,I reached out to Nina Kin about various feeds....,NaN,NaN,NaN,Expired Schedule Feed,NaN,recEEI0Hoj2x3HTTJ,LA Metro Rail
4,City of Bell: Los Angeles Metro feed to transi...,[recfPwQA6XD8SN2gR],Outreach,96,NaN,"{'id': 'usr2fo9oGEjsugXf0', 'email': 'evan@cal...",2023-09-16T00:18:07.000Z,No,1,1,OK,Yes,[7],Yes,2,2024,As of 9/14: Sent kickoff email to district rep...,2024-02-12,Waiting on District Rep,NaN,Los Angeles Metro feed transition,An issue to track migrating feeds off of the L...,recQIV7c2LReySeYp,La Campana


In [10]:
from calitp_data_analysis.sql import query_sql
full = query_sql("""
SELECT *
FROM `cal-itp-data-infra.mart_gtfs.fct_schedule_feed_downloads` fs
left join
`mart_transit_database.dim_provider_gtfs_data` dp
ON fs.gtfs_dataset_key = dp.schedule_gtfs_dataset_key
left join
`mart_transit_database.dim_services` ds 
ON dp.service_key = ds.key
left join
`mart_transit_database.bridge_services_x_operating_county_geographies` bs
ON ds.key = bs.service_key
left join
`mart_transit_database.dim_county_geography` dc
ON bs.county_geography_key = dc.key
WHERE
unzip_exception IS NOT NULL
OR
download_exception IS NOT NULL
OR
pct_files_successfully_parsed < 100
AND
dp._is_current=true
order by ts DESC 
LIMIT 100
""", as_df=True)
# issues
# issues

In [11]:
from calitp_data_analysis.sql import query_sql
issues = query_sql("""
SELECT ts
FROM `cal-itp-data-infra.mart_gtfs.fct_schedule_feed_downloads` fs
left join
`mart_transit_database.dim_provider_gtfs_data` dp
ON fs.gtfs_dataset_key = dp.schedule_gtfs_dataset_key
left join
`mart_transit_database.dim_services` ds 
ON dp.service_key = ds.key
left join
`mart_transit_database.bridge_services_x_operating_county_geographies` bs
ON ds.key = bs.service_key
left join
`mart_transit_database.dim_county_geography` dc
ON bs.county_geography_key = dc.key
WHERE
unzip_exception IS NOT NULL
OR
download_exception IS NOT NULL
OR
pct_files_successfully_parsed < 100
AND
dp._is_current=true
order by ts DESC 
LIMIT 100
""", as_df=True)
# issues
# issues

In [12]:
full.columns

Index(['key', 'feed_key', 'gtfs_dataset_key', 'ts', 'base64_url',
       'download_success', 'download_exception', 'last_modified_timestamp',
       'unzip_success', 'unzip_exception', 'zipfile_extract_md5hash',
       'zipfile_files', 'zipfile_dirs', 'pct_files_successfully_parsed',
       'key_1', 'public_customer_facing_fixed_route',
       'public_customer_facing_or_regional_subfeed_fixed_route',
       'organization_key', 'organization_name', 'organization_itp_id',
       'organization_hubspot_company_record_id', 'organization_ntd_id',
       'organization_source_record_id', 'service_key', 'service_name',
       'service_source_record_id', 'gtfs_service_data_customer_facing',
       'regional_feed_type', 'associated_schedule_gtfs_dataset_key',
       'schedule_gtfs_dataset_name', 'schedule_source_record_id',
       'service_alerts_gtfs_dataset_name', 'service_alerts_source_record_id',
       'vehicle_positions_gtfs_dataset_name',
       'vehicle_positions_source_record_id', 'trip_

In [13]:
issues

,ts
0,2024-02-15 03:00:34.470572+00:00
1,2024-02-15 03:00:34.470572+00:00
2,2024-02-15 03:00:34.470572+00:00
3,2024-02-15 03:00:34.470572+00:00
4,2024-02-15 03:00:34.470572+00:00
...,...
95,2024-02-13 03:00:50.708754+00:00
96,2024-02-13 03:00:50.708754+00:00
97,2024-02-13 03:00:50.708754+00:00
98,2024-02-13 03:00:50.708754+00:00


In [16]:
full

,key,feed_key,gtfs_dataset_key,ts,base64_url,download_success,download_exception,last_modified_timestamp,unzip_success,unzip_exception,zipfile_extract_md5hash,zipfile_files,zipfile_dirs,pct_files_successfully_parsed,key_1,public_customer_facing_fixed_route,public_customer_facing_or_regional_subfeed_fixed_route,organization_key,organization_name,organization_itp_id,organization_hubspot_company_record_id,organization_ntd_id,organization_source_record_id,service_key,service_name,service_source_record_id,gtfs_service_data_customer_facing,regional_feed_type,associated_schedule_gtfs_dataset_key,schedule_gtfs_dataset_name,schedule_source_record_id,service_alerts_gtfs_dataset_name,service_alerts_source_record_id,vehicle_positions_gtfs_dataset_name,vehicle_positions_source_record_id,trip_updates_gtfs_dataset_name,trip_updates_source_record_id,schedule_gtfs_dataset_key,service_alerts_gtfs_dataset_key,vehicle_positions_gtfs_dataset_key,trip_updates_gtfs_dataset_key,_valid_from,_valid_to,_is_current,key_2,source_record_id,name,service_type,mode,operating_counties,gtfs_schedule_status,gtfs_schedule_quality,manual_check__gtfs_realtime_data_ingested_in_trip_planner,manual_check__gtfs_schedule_data_ingested_in_trip_planner,deprecated_date,fixed_route,is_public,public_currently_operating,public_currently_operating_fixed_route,start_date,operational_status,_deprecated__currently_operating,_deprecated__assessment_status,_valid_from_1,_valid_to_1,_is_current_1,service_key_1,county_geography_key,service_name_1,county_geography_name,_valid_from_2,_valid_to_2,_is_current_2,key_3,source_record_id_1,name_1,fips,msa,caltrans_district,caltrans_district_name,place_geography,organization_key_1,service_key_2,_is_current_3,_valid_from_3,_valid_to_3
0,e02a0807e74e67924435dca420ecd589,None,611411d2519d4c3fd3aa70839ec46cdb,2024-02-15 03:00:34.470572+00:00,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,False,404 Client Error: Not Found for url: https://a...,None,None,None,None,[],[],None,ac58a8d6bda2f2b37657f9cf76ea2d60,False,False,81b4e86d203f5f57fb689fadcb652b56,San Joaquin Joint Powers Authority,None,8566807410,None,recvUlrKS1N2mvAwk,b3a26733da02debeb0e23e30c8d8e928,Amtrak San Joaquins,recGDwMYK8D3dhYWB,False,Regional Subfeed,611411d2519d4c3fd3aa70839ec46cdb,Bay Area 511 San Joaquins Schedule,recaPSZV7TH5X2Q3Y,None,None,Bay Area 511 San Joaquins VehiclePositions,reco734e15nOa2LJE,Bay Area 511 San Joaquins TripUpdates,rec2b5cQLMmrzeT2w,611411d2519d4c3fd3aa70839ec46cdb,None,e70c51eb9b687fa7da376bdfe76e8168,37f37a120a899c197c1b733003f9049b,2023-11-29 00:00:00+00:00,2023-12-08 23:59:59.999999+00:00,False,b3a26733da02debeb0e23e30c8d8e928,recGDwMYK8D3dhYWB,Amtrak San Joaquins,[fixed-route],[intercity rail],[San Joaquin],ok,2 - GTFS data has active service,Yes,No,None,True,Yes,True,True,None,Operating,True,True,2023-10-27 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,True,b3a26733da02debeb0e23e30c8d8e928,049397a4f78d12fe5e2c6414c1b0b21e,Amtrak San Joaquins,Contra Costa,2023-10-27 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,True,049397a4f78d12fe5e2c6414c1b0b21e,recPLw03qf2NQTzRU,Contra Costa,6013,OaklandBerkeleyLivermore,4,Bay Area / Oakland,"[recwgdY6JraWt1BLA, receFAJMkR8QNaIjT, recOcYA...","[recEEJVeGrHGoTwgj, recjnaKVDEgulsko3, recN5rq...","[recngbihqDmviGHgf, recmiu372RBid5LHN, recGDwM...",True,2022-06-29 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
1,e02a0807e74e67924435dca420ecd589,None,611411d2519d4c3fd3aa70839ec46cdb,2024-02-15 03:00:34.470572+00:00,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,False,404 Client Error: Not Found for url: https://a...,None,None,None,None,[],[],None,ac58a8d6bda2f2b37657f9cf76ea2d60,False,False,81b4e86d203f5f57fb689fadcb652b56,San Joaquin Joint Powers Authority,None,8566807410,None,recvUlrKS1N2mvAwk,b3a26733da02debeb0e23e30c8d8e928,Amtrak San Joaquins,recGDwMYK8D3dhYWB,False,Regional Subfeed,611411d2519d4c3fd3aa70839ec46cdb,Bay Area 511 San Joaquins Schedule,recaPSZV7TH5X2Q3Y,None,None,Bay Area 511 San Joaquins 